# State Student Spending Case Study
#### Examining any relationship between student sucess or lack thereof with how much money the state has invested


### Necessary Steps for Data Story
#### 1) Clean the Data 
    a. Import Data
    b. Transform variables
#### 2) Exploratory Data Analysis
    a. Visualize the data
#### 3) Preprocess and Train Data
#### 4) Create Machine Learning Model
#### 5) Summarize Findings

In [1]:
#import necessary modules

import pandas as pd
import numpy as np

# 1a. Import Data

In [2]:
#import Graduation rate, Dropout rate, State Spending, Population, & State GDP data
Grad = pd.read_csv('Data/Completion1318edit.csv')
Drop = pd.read_csv('Data/Dropout1318edit.csv')
StateSpend = pd.read_excel('Data/StateSpendingStudents.xlsx')
Pop = pd.read_csv('Data/Pop10-19.csv')
GDP = pd.read_csv('Data/State Quarterly GDP 2005 - 2021 - Sheet1.csv')

In [3]:
#import tax data 
Tax13 = pd.read_csv('Data/2013 State Taxes.csv')
Tax14 = pd.read_csv('Data/2014 State Taxes.csv')
Tax15 = pd.read_csv('Data/2015 State Taxes.csv')
Tax16 = pd.read_csv('Data/2016 State Taxes.csv')
Tax17 = pd.read_csv('Data/2017 State Taxes.csv')
Tax18 = pd.read_csv('Data/2018 State Taxes.csv')
Tax19 = pd.read_csv('Data/2019 State Taxes.csv')


#Note to self to automate process:
#Try a dictionary. Start with empty dictionary. Wild cards to read itieratively * .csv

## Cleaning Tax Data

In [4]:
#Examing the column names: Need to change them to match the rest of the data
Tax13.columns

Index(['Unnamed: 0', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC',
       'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
       'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VT', 'VA', 'WA', 'WV', 'WI', 'WY'],
      dtype='object')

In [5]:
#Dictionary to change state abbreviations to full state names
us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'District of Columbia': 'DC',
}

In [10]:
#Temporary reload of Tax Data
#import tax data 
Tax13 = pd.read_csv('Data/2013 State Taxes.csv')
Tax14 = pd.read_csv('Data/2014 State Taxes.csv')
Tax15 = pd.read_csv('Data/2015 State Taxes.csv')
Tax16 = pd.read_csv('Data/2016 State Taxes.csv')
Tax17 = pd.read_csv('Data/2017 State Taxes.csv')
Tax18 = pd.read_csv('Data/2018 State Taxes.csv')
Tax19 = pd.read_csv('Data/2019 State Taxes.csv')


#Note to self to automate process:
#Try a dictionary. Start with empty dictionary. Wild cards to read itieratively * .csv

# WHERE THE TROUBLE STARTS FOR TRYING TO CLEAN TAX*****

In [7]:
#Trying to make a function that will edit the files to the appropriate format
#Before I append them to a final file

def TaxEdit(df):
    df = df.rename(columns = us_state_abbrev)
    df = df.transpose()
    df = df.reset_index(drop = False)
    df.rename(columns = {'index': 'State'}, inplace = True)
    df = df.infer_objects() #I can't make it sum because I don't have dtype of numeric for my data
    return df

#Here is me attempting to force it to become numberic so I can sum
#The plan was to edit this. Try to make it a function or loop.
#I want to be able to sum the different tax categories save them as the appropriate year
#Then all frames will be cleaned, labeled with a year, and ready to append to final file.

Tax13Func = TaxEdit(Tax13)
Tax13E = Tax13Func.drop([0])
Tax13E.iloc[:,1:26] = Tax13E.iloc[:,1:26].astype('int64')
Tax13E['Total Tax'] =Tax13E.sum(axis = 1, skipna = True) 

Tax13E.head()

,State,0,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,24,Total Tax
1,Alabama,325652,2331676,174395,93,297958,530244,1557,737619,120110,...,14086,87908,4,3202520,382202,18,42155,119424,0,0.0
2,Alaska,99598,0,39194,8427,60236,41608,0,4295,69175,...,514,40522,8504,0,630941,0,0,4016966,0,0.0
3,Arizona,762651,6792635,68684,531,424369,781426,234,21013,315428,...,15747,126846,1973,3397707,662026,0,18902,29829,0,0.0
4,Arkansas,1022066,2837788,50656,36109,162962,455914,3113,0,237328,...,9917,121868,1649,2649577,402874,92,30190,80862,24623,0.0
5,California,1982208,33915885,357000,0,2242379,5492850,14088,676997,868703,...,416056,4202960,4027,66809000,7462000,0,0,37732,74978,0.0


## Below is how I was initially changing tax, but I decided to try the above to see if I could fix the data frames and add the year before appending the files.**** Not using code below

### Keeping the code below in case there is a way to append the files then add the yearly totals although I don't believe there is

In [11]:
TaxList = [Tax13, Tax14, Tax15, Tax16, Tax17, Tax18, Tax19]

#Append all the tax files into a single data frame.
TaxFinal = pd.DataFrame()

for df in TaxList:
    TaxFinal = TaxFinal[df].append(df)


#Rename State Abbreviations
TaxFinal = TaxFinal.rename(columns = us_state_abbrev)

## NEW NOTES
# First Step: Transpose the data as well as reset the index   

TaxFinal = TaxFinal.transpose()
TaxFinal = TaxFinal.reset_index(drop = False)
TaxFinal.rename(columns = {'index': 'State'}, inplace = True)

# Second step: Group by state
#TaxFinal = TaxFinal.groupby(['State'])

#Find Total Taxes
TaxFinal['Total Taxes'] = TaxFinal.sum(axis = 1, skipna = True)


#THIS IS INCORRECT

#Then use melt function on this single data frame.
#TaxFinal = TaxFinal.melt(id_vars = ['State'], var_name='Year', value_name = 'Taxes')

#Examine
print(TaxFinal.head())

ValueError: Boolean array expected for the condition, not object

## Cleaning Median Income Data

In [ ]:
#Read in Median Income (Deleted out Standard Deviation as well as repetitive information from original file)
MedianInc = pd.read_csv('Data/Median Income Edit.csv')

#Flip
MedianInc = MedianInc.melt(id_vars = ['State'], var_name='Year', value_name = 'MedInc')

MedianInc.head()
MedianInc.tail()
#Delete unnecessary columns and repetitive information.

## Cleaning Graduation and Drop Out Data

In [ ]:
#examine data
print(Grad.columns)

In [ ]:
#Rename columns so they match years in other files.
oldname = Grad.columns
newname = ['State', 2013, 2014, 2015, 2016, 2017, 2018 ]
Grad = Grad.rename(columns=dict(zip(oldname, newname)))

#Flip so States are columns
Grad = Grad.melt(id_vars = ['State'], var_name='Year', value_name = 'Graduates')


#Examine
Grad.head()

In [ ]:
#examine data
print(Drop.head())
print(Drop.columns)

In [ ]:
#Flip so States are columns
Drop = Drop.melt(id_vars = ['State'], var_name='Year', value_name = 'Dropout')

#Examine
Drop.head()

## Clean State Spending Data

In [ ]:
#Examine State Spending
print(StateSpend.head())

In [ ]:
#Realign with States as columns
StateSpend.rename(columns = {'state': 'State'}, inplace = True)
StateSpend = StateSpend.melt(id_vars = ['State'], var_name='Year', value_name = 'StateSpend')
StateSpend.head()

#Examine
print(StateSpend.head())

## Clean Population Data

In [ ]:
#Examine the Population
print(Pop.head(10))
print(Pop['Geographic Area'].head(10))
#print(Pop.rows)

In [ ]:
#Fix geographic area names
Pop['Geographic Area'] = Pop['Geographic Area'].str.replace('.','')
print(Pop['Geographic Area'].head(10))

In [ ]:
#Drop the US regions
Pop = Pop.drop([0,1,2,3,4,])
Pop = Pop.reset_index(drop = True)
Pop.tail(10)

In [ ]:
#Flip Columns
Pop.rename(columns = {'Geographic Area': 'State'}, inplace = True)
Pop = Pop.melt(id_vars = ['State'], var_name='Year', value_name = 'Population')
Pop.head()

## Clean GDP  Data

## Not sure how to extract the year***

### Trying to move the columns frame down so that I can change the dtype and extract year
But am getting confused **

In [ ]:
import pandas as pd
GDP = pd.read_csv('Data/State Quarterly GDP 2005 - 2021 - Sheet1.csv')

#Flip states to a column.
GDP = GDP.transpose()


#Trying to move states

GDP.columns = GDP.iloc[0,:]
GDP = GDP.iloc[1:, :]

#Need to rename columns
GDP = GDP.reset_index()

#Properly name State column
GDP = GDP.rename({'index': 'State'}, axis='columns')

GDP.head()

# Then use melt function
#GDPmelt = GDP.melt(id_vars = ['State'], var_name='QTR Year', value_name = 'State GDP')
#GDPmelt.head()

#Take average across 4 quarters for yearly GDP
### THIS CLEARLY WON'T WORK BECAUSE OF DTYPE ISSUES AND ETC, but this is where my mind is stuck

#for x in GDP.columns:
    #year = x.year
    #print(year)
    
#Use year extractor
#Add max/min/median as well to round it out

## Clean Unemployment Rates

In [ ]:
# Empty data frame
final_unemployment_data = pd.DataFrame()


for i in np.arange(2009,2018,1):
    print("Reading: {}".format(str(i)))
    
    year_data = pd.read_excel('Data/UnemploymentRateData_Parsed.xlsx', sheet_name = str(i))
    melt_data = pd.melt(year_data, id_vars = 'State', var_name = ['Month'], value_name = 'value')

    aggregated_data = melt_data.groupby('State').agg({'value' : ['min', 'max', 'mean']}).reset_index()
    aggregated_data.columns = ['State', 'min_unemploymentRate', 'max_unemploymentRate', 'mean_unemploymentRate']
    aggregated_data['Year'] = str(i)
    
# Logic to add each years data to the end of previously created data
final_unemployment_data = final_unemployment_data.append(aggregated_data)

final_unemployment_data.head()

## Merge ALL the clean files

In [ ]:
#Change them all to strings because??

Grad['Year'] = Grad['Year'].astype('str')
Grad['State'] = Grad['State'].astype('str')

Drop['Year'] = Drop['Year'].astype('str')
Drop['State'] = Drop['State'].astype('str')

StateSpend['Year'] = StateSpend['Year'].astype('str')
StateSpend['State'] = StateSpend['State'].astype('str')

Pop['Year'] = Pop['Year'].astype('str')
Pop['State'] = Pop['State'].astype('str')

In [ ]:
#merge graduate and drop data
grad_drop_merge = Grad.merge(Drop, on = ['Year', 'State'], how = 'left')
grad_drop_merge.head()

In [ ]:
#Merge state spending data
all_spend_merge = grad_drop_merge.merge(StateSpend, on = ['Year', 'State'], how = 'left')
all_spend_merge.head()

In [ ]:
# Merge in population
all_merge = all_spend_merge.merge(Pop, on = ['Year', 'State'], how = 'left')
all_merge.head()

# Begin Exploratory Data Analysis

In [ ]:
#Limit to years of interest
#StateSpendTrain = StateSpend.iloc[16:20]

#State Spend Documentation ends after 2016
#StateSpendTest = StateSpend.iloc[19:]

#print(StateSpendTrain)
#print(StateSpendTest)